In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, KFold


In [ ]:
df = pd.read_csv('dataset_wiremind.csv')
df.head()

## Checking for Missing Values
We need to confirm that there are no missing values in the dataset before proceeding with further analysis. This ensures that we do not run into errors during the modeling process.

In [ ]:
df.isnull().sum()

No missing values, great! So let's start with the EDA.


In [ ]:
df.dtypes

# EDA

In [ ]:
# Plotting basic distributions to better understand whats inside the dataset 
numerical_columns = ['ChargeableWeight', 'Revenue', 'Pieces']
for col in numerical_columns:
    plt.figure(figsize=(10, 6))
    plt.hist(df[col], bins=30, color='skyblue', edgecolor='black')
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

# Plotting the top 10 categorical feature distributions
categorical_columns = ['AgentCode', 'ProductCode']
for col in categorical_columns:
    plt.figure(figsize=(12, 6))
    df[col].value_counts().nlargest(10).plot(kind='bar', color='lightgreen')
    plt.title(f'Frequency of {col} (Top 10)')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.show()


In [ ]:
df.describe().round(0).astype(int)

Generated a summary of the dataset with rounded values to get an overview of the central tendency and distribution without decimals.


Checking the column names in the dataset to understand the available features.


In [ ]:
df.columns

Counting the number of unique values for each column to identify categorical features and assess the variability of the data.


In [ ]:
for i in df.columns:
    print(i, len(df[i].unique()))

##### we can see that we have only 1 year for FlownYear (2017), 1 destination for DestinationCode so we can remove them because they won't help our model 

In [ ]:
#create a copy of the original DataFrame to preserve the original data
new_df = df.copy()
#drop unnecessary columns that are not needed for analysis
new_df = new_df.drop(["DocumentRatingSource","FlownYear","DestinationCode" ], axis =1)
new_df.head()

In [ ]:
#calculate the target variable as revenue per ChargeableWeight
new_df["target"] = new_df["Revenue"]/ new_df["ChargeableWeight"]
#display the first few rows to verify the new 'target' column
new_df.head()

In [ ]:
new_df.describe().round(0).astype(int)

Summary statistics of the modified dataset to assess the updated features and new target variable.


### MONTHS ANALYSIS: 

Calculating the average Chargeable Weight, Revenue, and Target for each month to understand trends over time.


In [ ]:
month_avg = new_df.groupby('FlownMonth')[["ChargeableWeight", 'Revenue', 'target']].mean()
month_avg

In [ ]:
#count the number of shipments for each month
monthly_shipments = new_df['FlownMonth'].groupby(df['FlownMonth']).size()
print(monthly_shipments)
#calculate the percentage of shipments for each month
monthly_percentage = (monthly_shipments / len(new_df)) * 100

monthly_percentage.round(3)

The output shows that the distribution of shipments across different months is relatively balanced, with each month contributing between 31.49% and 34.38% of the total shipments. This suggests that there is no **major** seasonality effect in shipment frequency.


Visualizing the average revenue and average revenue per kg (target) by month to identify any trends or seasonality effects.


In [ ]:
metrics = ['FlownMonth', 'Revenue', 'Pieces']
titles = ['Total Shipments by Flown Month', 'Total Revenue by Flown Month', 'Average Number of Pieces Shipped by Month']
y_labels = ['Total Shipments', 'Total Revenue', 'Average Pieces']
colors = ['g', 'b', 'r']

plt.figure(figsize=(15, 6))

for i, metric in enumerate(metrics):
    plt.subplot(1, 3, i + 1)
    if metric == 'FlownMonth':
        # Count the number of shipments per month for the first graph
        df_grouped_by_month = df.groupby('FlownMonth').size()
    elif metric == 'Pieces' and i == 2:
        # Calculate the average number of pieces shipped by month for the third graph
        df_grouped_by_month = df.groupby('FlownMonth')[metric].mean()
    else:
        df_grouped_by_month = df.groupby('FlownMonth')[metric].sum()
    
    df_grouped_by_month.plot(kind='line', marker='o', color=colors[i])
    plt.title(titles[i])
    plt.xlabel('Month')
    plt.ylabel(y_labels[i])
    plt.grid(True)

plt.tight_layout()  
plt.show()


### Total Shipments and Revenue Trends:
November shows the highest total number of shipments and revenue, followed by October, with September having the lowest figures. This indicates that November might be a peak period, potentially due to seasonal demand or promotional activities. 

We can also see that the average number of pieces shipped by month is the lowest in october, we will have to conduct another analysis to see if the number of pieces has a role to play with the total weight of each shipment 


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 6))
monthly_weight_mean = new_df.groupby('FlownMonth')['ChargeableWeight'].mean()
#plot for average revenue by month
ax[0].bar(month_avg.index, month_avg['Revenue'], color='skyblue')
ax[0].set_title('Average Revenue by Month')
ax[0].set_ylabel('Average Revenue')

#plot for average revenue per kg (target = revenue/chargeable weight)
ax[1].bar(month_avg.index, month_avg['target'], color='orange')
ax[1].set_title('Average Revenue per kg (Target) by Month')
ax[1].set_ylabel('Average Revenue per kg')

ax[2].bar(monthly_weight_mean.index, monthly_weight_mean, color='green')
ax[2].set_title('Average Chargeable Weight by Month')
ax[2].set_ylabel('Average Chargeable Weight (kg)')

plt.tight_layout()
plt.show()

### Global Analysis of Shipment and Revenue Trends

The visualizations provide an initial overview of the shipment and revenue patterns across the months of September, October, and November. Here's a summary of the observations:


   
insights:

1. **Average Revenue and Revenue per kg**:
   - The average revenue per shipment is also the highest in November, aligning with the total revenue pattern. However, the average revenue per kg in November is lower compared to October and September. 

2. 
we ship fewer pieces in october (previous cell 3rd plot) but with the new graphs we can see that the average revenue per kg is the highest in october and that the average weight per shipment is also higher in october which might mean that in october they ship heavier items (pieces) that are more expensive.

3. **This could indicate that shipments in November are larger in volume but involve lower pricing per unit weight.**


### Next Steps
To gain deeper insights into these patterns, further analysis will be conducted focusing on:
   - **Cargo Type**: Understanding which cargo types dominate during these months and how they impact overall revenue and efficiency.
   - **Product Type**: Exploring the specific products shipped and their influence on revenue per kg and shipment volume.
   - **Delivery Agents**: Investigating how different delivery agents contribute to these patterns, which could reveal whether certain agents specialize in specific types of cargo or regions.

These additional dimensions will help refine the analysis and provide more targeted explanations for the observed trends.


### CARGO TYPE ANALYSIS: 

In [ ]:
# Create the plot
plt.figure(figsize=(16, 8))
ax = sns.countplot(data=df, x='FlownMonth', hue='CargoType', palette='viridis')

# for loop to exactly see the number of different type of cargotype shipment made per month
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'baseline', 
                fontsize=12, color='black', xytext = (0, 5), 
                textcoords = 'offset points')

#adding titles and labels
plt.title('Number of Shipments per Month by CargoType')
plt.xlabel('Month')
plt.ylabel('Number of Shipments')
plt.legend(title='CargoType')
plt.tight_layout()

# Show the plot
plt.show()

# Grouping the dataset by cargotype to calculate the average target for each type
average_target_per_cargo = new_df.groupby('CargoType')['target'].mean().sort_values()

# Plotting the average target per cargo type
plt.figure(figsize=(15, 6))
average_target_per_cargo.plot(kind='barh', color='skyblue')
plt.title('Average Revenue per kg (Target) by Cargo Type')
plt.xlabel('Average Revenue per kg')
plt.ylabel('Cargo Type')
plt.show()


### Link Between Shipment Volume and Revenue per kg by Cargo Type

The two graphs provide complementary insights into the shipment patterns and the revenue efficiency per kilogram across different cargo types. Here's how they are connected:

1. **Shipment Volume by Cargo Type**:
   - The first graph shows the number of shipments per month for each cargo type. It is evident that **ZZZ** dominates the shipment volume in all months, with significantly higher counts compared to **YYY** and **XXX**. This suggests that **ZZZ** is the most commonly shipped cargo type, contributing the most to the overall shipment count.

2. **Revenue Efficiency per kg by Cargo Type**:
   - The second graph displays the average revenue per kilogram for each cargo type. Interestingly, **YYY** has the highest revenue per kilogram, while **ZZZ**, despite its dominance in volume, has a lower revenue efficiency. **XXX** also shows lower revenue per kg compared to **YYY**.

3. **Interpretation**:
   - The relationship between these graphs indicates that while **ZZZ** is the most frequently shipped, it is not the most profitable per unit weight. The high volume of **ZZZ** shipments likely targets bulk orders or less valuable products, which explains its lower revenue per kg. 
   - On the other hand, **YYY**, although shipped in smaller quantities, commands a much higher revenue per kilogram **(2.75 times more expensive per kilogram than ZZZ and 3.67 times more expensive per kilogram than XXX)**. 
   
This suggests that **YYY** might be reserved for high-value or premium products, which justify the higher pricing. 


### Product Type analysis:

In [ ]:
# Bar chart for average Price_per_kg by ProductCode
plt.figure(figsize=(15, 5))

# Calculate the mean price per kg for each product code
product_code_avg_price = new_df.groupby('ProductCode')['target'].mean().sort_values(ascending=False)

# Plot the bar chart
ax = product_code_avg_price.plot(kind='bar', color='lightgreen')
plt.title('Average Price per kg by ProductCode')
plt.ylabel('Average Price per kg')
plt.xticks(rotation=90)
plt.tight_layout()

# Add the numbers on top of the bars
for index, value in enumerate(product_code_avg_price):
    plt.text(index, value, f'{value:.2f}', ha='center', va='bottom')

# Display the plot
plt.show()





The **average target value (revenue per kg)** for each product code shows significant variation across different product types:

- The product code **XGH** has the highest average target value of 285, which is much greater than other products. This suggests that **XGH** may be an outlier, potentially biasing our analysis.
- Other product codes, such as **SJD** and **X**, also have relatively high average target values, while **PCS** has the lowest.
- The substantial difference between the highest and lowest average target values suggests that certain products may disproportionately impact the overall revenue per kg.

To ensure the robustness of our model, it might be worth considering whether the extreme values, such as **XGH**, should be treated as outliers and possibly removed or further investigated.


In [ ]:
#counting the number of time we have the productcode XGH in our dataset 
# This helps identify how many times this outlier (product code "XGH") is repeated
len(new_df.loc[new_df['ProductCode'] == "XGH"])

In [ ]:
# Filter to keep only the top 10 most shipped ProductCodes
top_10_products = df['ProductCode'].value_counts().nlargest(20).index
filtered_df = df[df['ProductCode'].isin(top_10_products)]
plt.figure(figsize=(20, 10))
ax = sns.countplot(data=filtered_df, x='FlownMonth', hue='ProductCode', palette='tab10')

plt.title('Distribution of Top 10 ProductCodes per Month')
plt.xlabel('Month')
plt.ylabel('Count of Product Shipped')
plt.xticks(rotation=0)
plt.legend(title='ProductCode', bbox_to_anchor=(1.05, 1), loc='upper left')

# Annotating each bar with the count
for p in ax.patches:
    height = p.get_height()
    if height > 0:  # Only annotate if there is a count
        ax.text(
            p.get_x() + p.get_width() / 2.,  # x position: center of the bar
            height + 1,  # y position: just above the bar
            f'{height}',  # the value to display
            ha='center', va='bottom', fontsize=9, color='black'
        )

plt.tight_layout()
plt.show()


revenue_per_product_per_month = new_df.groupby(['ProductCode', 'FlownMonth'])['Revenue'].sum().reset_index()

# Find the top 10 product codes with the highest total revenue across all months
top_10_products = revenue_per_product_per_month.groupby('ProductCode')['Revenue'].sum().nlargest(10).index

# Filter the dataset to include only the top 10 product codes
top_10_revenue_per_product_per_month = revenue_per_product_per_month[revenue_per_product_per_month['ProductCode'].isin(top_10_products)]

plt.figure(figsize=(12, 6))
sns.barplot(data=top_10_revenue_per_product_per_month, x='FlownMonth', y='Revenue', hue='ProductCode')
plt.title('Revenue by Top 10 Product Codes for Each Month')
plt.xlabel('Month')
plt.ylabel('Total Revenue')
plt.legend(title='ProductCode', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


average_weights = df.groupby('ProductCode')['ChargeableWeight'].mean().reset_index()

# Sort the average weights in descending order (heaviest to lightest)
average_weights = average_weights.sort_values(by='ChargeableWeight', ascending=False)

# Plotting the average weight for each product code
plt.figure(figsize=(10, 6))
plt.bar(average_weights['ProductCode'], average_weights['ChargeableWeight'], color='skyblue')
plt.xlabel('Product Code')
plt.ylabel('Average Weight')
plt.title('Average Weight per Product Code (Heaviest to Lightest)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


The plots above reveal key insights into product performance across different months:

**Distribution of Top 10 Product Codes per Month:**
- The product code **DLJ** is consistently the most shipped product across all months, with significantly higher shipment counts compared to other products.
- Despite DLJ’s volume dominance, it does not generate the highest revenue, indicating it may be a lower-value or bulk product.

**Revenue by Top 10 Product Codes for Each Month:**
- The product code **MGK**, although shipped far less frequently than DLJ, generates the most revenue across all months 
ps: MGK 48.91% more expensive than DLJ [((13.62-9.15)/9.15)  *100] (we get the information from the graph showing the average price per kg per product code )


These observations highlight that while DLJ dominates in terms of shipment volume but MGK is the primary driver of revenue. This contrast suggests that different strategies may be needed to manage each product type effectively.

### Agents shipping Analysis: 

#### plot the top 30 agents with the highest average price per kg 

In [ ]:
agent_impact = new_df.groupby('AgentCode')['target'].mean().sort_values(ascending=False)
#print(agent_impact.head(10))
plt.figure(figsize=(10, 5))
agent_impact[:30].plot(kind='bar', color='green')
#titles and labels for better readability 
plt.title('average charge per kilogram by agent that owns the shipment')
plt.xlabel('Month')
plt.show()
plt.figure(figsize=(12, 6))
sns.swarmplot(x='target', y='AgentCode', data=new_df, order=agent_impact[:15].index, palette='deep')
plt.title('Individual Charges per Kilogram by Agent')
plt.xlabel('Charge per Kilogram')
plt.ylabel('Agent Code')
plt.show()

### Analysis of Agent Charges per Kilogram

The two visualizations provide insight into the charging patterns of various agents and how they impact the overall pricing strategy:

1. **Average Charge per Kilogram by agent**:
   - This graph shows the average charge per kilogram for each agent. It is clear that agent **GIKGZVO** has an exceptionally high average charge compared to other agents. This could indicate a specialization in high-value or premium cargo, leading to higher per kilogram rates.
   - Other agents show significantly lower average charges, suggesting they handle more standard or bulk cargo that doesn’t command high pricing.

2. **Individual Charges per Kilogram by agent**:
   - The scatter plot reveals the distribution of individual charges per kilogram for each agent. It shows that while most agents maintain a relatively consistent and lower charge range, agent **GIKGZVO** has a few extreme high values, confirming its higher average rate.
   - This variability indicates that some agents might have a mixed strategy, handling both high-value and standard shipments, while others stick to a consistent low-cost approach.
   
   
**With GIKGZVO having a significantly higher value compared to other agents, it can be considered an outlier. Removing it from the dataset could potentially improve the model's performance by reducing the influence of extreme values, leading to a more generalized model that better represents the majority of the data**

In [ ]:
# Group by AgentCode and FlownMonth to count shipments
shipment_count_per_agent = new_df.groupby(['AgentCode', 'FlownMonth']).size().unstack(fill_value=0)

# Find the top 10 agents with the highest total shipments
top_10_agents_by_shipment = shipment_count_per_agent.sum(axis=1).nlargest(10).index

# Filter the dataset to include only the top 10 agents by shipment count
top_10_shipment_count = shipment_count_per_agent.loc[top_10_agents_by_shipment]

plt.figure(figsize=(10, 8))
sns.heatmap(top_10_shipment_count, cmap='YlGnBu', annot=True, fmt='d')
plt.title('Total Shipments per Top 10 Agents by Month')
plt.xlabel('Month')
plt.ylabel('Agent Code')
plt.show()

The heatmap visualizes the total shipments per month for the top 10 agents, showing a clear pattern:

The agent VFZB6JC consistently leads in all three months (September, October, and November), making it the most important or dominant shipper among the agents. The number of shipments for VFZB6JC remains significantly higher than the other agents, indicating its prominent role in shipment volume throughout the period. This highlights VFZB6JC as a key player in the shipping network.

In [ ]:

# Filter the dataset to include only the top 10 agents based on 'agent_impact'
top_agents = agent_impact[:20].index
filtered_df = new_df[new_df['AgentCode'].isin(top_agents)]

# Plot 1: Types of products shipped by these agents
plt.figure(figsize=(12, 6))
sns.countplot(y='ProductCode', data=filtered_df, order=filtered_df['ProductCode'].value_counts().index, hue='AgentCode', palette='viridis')
plt.title('Product Types Shipped by Top Agents')
plt.xlabel('Count')
plt.ylabel('Product Code')
plt.legend(title='Agent Code', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# Plot 2: Types of cargo used by these agents
plt.figure(figsize=(12, 6))
sns.countplot(y='CargoType', data=filtered_df, order=filtered_df['CargoType'].value_counts().index, 
              hue='AgentCode', palette='coolwarm')

plt.title('Cargo Types Used by Top Agents')
plt.xlabel('Count')
plt.ylabel('Cargo Type')
plt.legend(title='Agent Code', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# Filtering the dataset to include the top 15 agents based on 'agent_impact'
top_15_agents = agent_impact[:10].index
filtered_df_top_15 = new_df[new_df['AgentCode'].isin(top_15_agents)]
# Count the number of unique product types each agent ships
product_count_per_agent = filtered_df_top_15.groupby('AgentCode')['ProductCode'].nunique().sort_values(ascending=False)

# Plotting the adjusted bar plot
plt.figure(figsize=(12, 8))
product_count_per_agent.plot(kind='barh', color='purple')
plt.title('Number of Product Types Shipped by Top 15 Agents')
plt.xlabel('Number of Product Types')
plt.ylabel('Agent Code')
plt.gca().invert_yaxis()  
plt.show()


### Analysis of Agent HNHR0VO and Product MGK

- **HNHR0VO** exclusively ships **MGK**, which is in the **top 10 most expensive products**.
- HNHR0VO ranks among the **top 10 most expensive agents**, likely due to handling only high-value shipments.
- **DLJ** is the most shipped product but generates less revenue compared to MGK, reflecting its bulk/low-value nature.
- **SJD** follows a similar trend, being shipped by multiple agents with lower specialization and costs.

**Key Insight**: HNHR0VO's premium pricing aligns with MGK's high value, while DLJ's bulk strategy shows volume but less revenue impact.



In [ ]:
plt.figure(figsize=(7, 7))
sns.heatmap(new_df[['ChargeableWeight', 'Pieces', 'Revenue', 'target']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

The correlation heatmap shows that the current features do not have strong enough correlations to accurately predict the target, except for a moderate correlation with revenue. Therefore, feature engineering may be necessary to identify better predictors.


# Feature engineering:

We are creating two new features as part of **feature engineering**:

1. **Agent_Avg_Revenue**: This feature represents the average revenue generated by each agent. By including the average revenue per agent, we can capture the overall contribution of an agent to the revenue. This could help identify whether agents with higher average revenues have a consistent impact on pricing or profit margins, providing valuable insights into agent performance.

2. **Agent_Total_Weight**: This feature represents the total chargeable weight managed by each agent. Adding the total weight helps us understand the scale of operations for each agent, which may influence the costs and pricing strategies. Agents handling more weight might benefit from economies of scale, which could impact the model.

These engineered features will help provide a more nuanced understanding of each agent's impact on the target variable, potentially improving model performance by capturing relationships between agent-level metrics and shipment pricing.


In [ ]:
agent_avg_revenue = new_df.groupby('AgentCode')['Revenue'].transform('mean')
new_df['Agent_Avg_Revenue'] = agent_avg_revenue

agent_total_weight = new_df.groupby('AgentCode')['ChargeableWeight'].transform('sum')
new_df['Agent_Total_Weight'] = agent_total_weight

In [ ]:
# Average price per kg for each ProductCode
product_avg_price_per_kg = new_df.groupby('ProductCode')['target'].transform('mean')
new_df['Product_Avg_Price_per_kg'] = product_avg_price_per_kg

In [ ]:
label_encoder_agent = LabelEncoder()
label_encoder_product = LabelEncoder()
label_encoder_flown = LabelEncoder()
label_encoder_cargo = LabelEncoder()
# Encode categorical features into numerical ones
new_df['AgentCode_encoded'] = label_encoder_agent.fit_transform(new_df['AgentCode'])
new_df['ProductCode_encoded'] = label_encoder_product.fit_transform(new_df['ProductCode'])
new_df['FlownMonth_encoded'] = label_encoder_flown.fit_transform(new_df['FlownMonth'])
new_df['CargoType_encoded'] = label_encoder_cargo.fit_transform(new_df['CargoType'])

# Checking numerical columns after feature engineering
numerical_columns = new_df.select_dtypes(include=['number']).columns
print(pd.DataFrame(numerical_columns))


The new feature Product_Avg_Price_per_kg  provides the average price per kg for each product code helping us understand the typical revenue generated per unit weight for different products.


## Preparing the data for our model:

We are preparing the dataset for modeling by performing several important preprocessing steps. This includes removing outliers, which could bias the model, and encoding categorical features so they can be used effectively in a machine learning model.


In [ ]:
# Make a copy of the dataset in case we need to preserve the original data
dataset = new_df.copy()
datasetout = new_df.copy()
# Drop rows with outliers that could bias the model due to their large deviation from the mean
dataset = dataset.drop(dataset[dataset['AgentCode'] == "GIKGZVO"].index)

This is the **second correlation heatmap** we are creating, focusing on evaluating the impact of the newly engineered features alongside the original features. By including newly created variables like `Agent_Avg_Revenue`, `Agent_Total_Weight`, and encoded categorical features, we aim to observe whether these additions bring stronger correlations with the **target** variable. This analysis will help us determine if the new features are likely to be **important predictors** and thus enhance our model's performance.


In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(dataset[['ChargeableWeight', 'Pieces', 'Revenue', 'Agent_Avg_Revenue',
       'Agent_Total_Weight', 'Product_Avg_Price_per_kg','AgentCode_encoded', 
                     'ProductCode_encoded','FlownMonth_encoded', 'CargoType_encoded','target']].corr(), 
            annot=True, 
            cmap='inferno')

plt.title('Correlation Heatmap')
plt.show()

The newly engineered features, **`Agent_Avg_Revenue`** and **`Product_Avg_Price_per_kg`**, seem to add value in predicting the target as they both exhibit relatively strong correlations (`0.69` and `0.17`, respectively).

The encoded categorical features (**`AgentCode_encoded`**, **`ProductCode_encoded`**, **`FlownMonth_encoded`**) show **low correlation** with the target, meaning their direct impact is limited. However, these encoded features can still be useful for the model to identify relationships that might not be captured through simple correlation.

The heatmap helps us understand that **feature engineering** added more explanatory power, especially with **`Agent_Avg_Revenue`**, and it will be an important factor to consider during model training.


In [ ]:
#not including revenue and cheargeable weight in X to avoid data leakeage 
X = dataset[['AgentCode_encoded', 'ProductCode_encoded', 'ChargeableWeight', 'Pieces', 'FlownMonth_encoded','Agent_Avg_Revenue', 'Agent_Total_Weight', 'Product_Avg_Price_per_kg', 'CargoType_encoded']]





#in the next line of code we removed Agent_avg_revenue to see how worse our models were going to be
#without this feature
#and yes it indeed lowered our model performance by about 20%
#at the end i still decided to keep it, but feel free to remove the # and re run the code to see 
#the difference without it on our models performance
#X = dataset[['AgentCode_encoded', 'ProductCode_encoded', 'ChargeableWeight', 'Pieces', 'FlownMonth_encoded', 'Agent_Total_Weight', 'Product_Avg_Price_per_kg', 'CargoType_encoded']]

y = dataset['target']

In [ ]:
#X and y but with the outlier GIKGZVO
X_out = datasetout[['AgentCode_encoded', 'ProductCode_encoded', 'ChargeableWeight', 'Pieces', 'FlownMonth_encoded', 'Agent_Avg_Revenue', 'Agent_Total_Weight', 'Product_Avg_Price_per_kg', 'CargoType_encoded']]
        
#in the next line of code we removed Agent_avg_revenue to see how worse our models were going to be
#without this feature
#and yes it indeed lowered our model performance by about 20%
#at the end i still decided to keep it, but feel free to remove the # and re run the code to see 
#the difference without it on our models performance
                    
#X_out = datasetout[['AgentCode_encoded', 'ProductCode_encoded', 'ChargeableWeight', 'Pieces', 'FlownMonth_encoded', 'Agent_Total_Weight', 'Product_Avg_Price_per_kg', 'CargoType_encoded']]
y_out = datasetout['target']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.27, 
                                                random_state=42)

In [ ]:
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(X_out, y_out, test_size=0.27, 
                                                random_state=42)

In [ ]:
#importing the necessary libraries for our models 
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

### Model Selection

We chose **Gradient Boosting Regressor**, **XGBoost Regressor**, and **Random Forest Regressor** due to the nature of our problem: predicting a continuous numeric value (regression task). These models are well-suited for regression because:

- **Gradient Boosting Regressor** and **XGBoost**:
  - Utilize boosting techniques to sequentially improve weak learners, creating a strong predictive model.
  - Effectively capture complex, non-linear relationships in the data.
  - Achieve high accuracy on complex datasets through optimization of loss functions.

- **Random Forest Regressor**:
  - Builds multiple decision trees and averages their predictions, reducing overfitting.
  - Handles feature interactions well and provides a robust, ensemble-based solution.

By comparing these models, we aim to identify the most effective one for our regression problem based on their performance.


In [ ]:
#importing the necessary libraries for model validation
from sklearn.metrics import mean_squared_error,mean_absolute_error ,r2_score

### Evaluation Metrics for Regression Models

In evaluating the performance of regression models such as `GradientBoostingRegressor`, `XGBoost`, and `RandomForestRegressor`, it's crucial to choose metrics that provide meaningful insights into the model's predictive capabilities. The following metrics were selected:

---

#### 1. R-squared (R²)

- **Definition**: R² measures the proportion of variance in the dependent variable that is predictable from the independent variables.
- **Why Choose R²?**
  - **Interpretability**: It provides a clear indication of how well the independent variables explain the variability of the target variable.
  - **Model Fit**: An R² value closer to 1 suggests a better fit, indicating that the model explains a large portion of the variance.
  - **Comparative Analysis**: Useful for comparing the explanatory power of different models.

---

#### 2. Root Mean Squared Error (RMSE)

- **Definition**: RMSE is the square root of the average of squared differences between the predicted and actual values.
- **Why Choose RMSE?**
  - **Scale Sensitivity**: RMSE is in the same units as the target variable, making it easy to interpret the magnitude of errors.
  - **Error Magnitude**: It penalizes larger errors more than smaller ones due to the squaring of differences, which is helpful when large errors are particularly undesirable.
  - **Model Optimization**: Commonly used as a loss function for regression models, aiding in model tuning.

---



####  3. Mean Absolute Error (MAE)

- **Definition**: MAE calculates the average magnitude of errors between predicted and actual values, without considering their direction.
- **Why MAE is Useful:**
  - **Robustness to Outliers**: Unlike RMSE, MAE does not square the errors, so it is less sensitive to outliers.
  - **Interpretability**: Provides a straightforward average error magnitude, which is easy to understand.
  - **Balanced Error Perspective**: Offers a different viewpoint on model performance by treating all errors equally.


## Evaluation Using train test split method

In [ ]:
import time

In [ ]:

# Function to train and evaluate a given model and return performance metrics without having to redo it for each model 
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    # Start measuring training time
    start_time = time.time()  
    
    # Train the provided model on training data
    model.fit(X_train, y_train)  
    
    # Calculate the total time taken to train the model
    train_time = time.time() - start_time  
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)  

    # Calculate evaluation metrics: 
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Return the results as a dictionary for easy appending to the summary list
    return {
        'Model': model_name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'Training Time (s)': train_time
    }

# Initialize an empty list to store the evaluation results of each model
results = []

# Example models to evaluate
models = [
    ('XGBoost', xgb.XGBRegressor(n_estimators=450, max_depth=4, learning_rate=0.1, random_state=42)),
    ('Random Forest', RandomForestRegressor(n_estimators=500, max_depth=6, random_state=42)),
    ('Gradient Boosting', GradientBoostingRegressor(n_estimators=450, max_depth=4, learning_rate=0.1, random_state=42))
]

# Evaluate each model on both datasets
for model_name, model in models:
    # Evaluate on the dataset without the outlier
    result = train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name + ' (No Outlier)')
    results.append(result)
    
    # Evaluate on the dataset with the outlier
    result_out = train_and_evaluate_model(model, X_train_out, y_train_out, X_test_out, y_test_out, model_name + ' (With Outlier)')
    results.append(result_out)

# Convert results to a DataFrame for easier visualization
results_dfi = pd.DataFrame(results)


We are defining a reusable function that takes a model, trains it, evaluates it, and returns the results. This function allows us to apply the same process consistently across different models while avoiding repetitive code. By using this function, we can easily track key metrics—such as training time, RMSE, MAE and R2 score**—for each model, and store these results in an organized way for later comparison. The use of an empty list (`results`) ensures that we can gather the metrics from each model to create a summary DataFrame that facilitates easy model evaluation and selection.


## Evaluation Using Cross-Validation
To enhance the robustness of our evaluation, we now use cross-validation (with 5 folds) instead of a simple train-test split. This approach helps verify the stability and reliability of the models' performance.


In [ ]:
# Function to train and evaluate a given model using cross-validation and return performance metrics
def train_and_evaluate_model_cv(model, X, y, cv, model_name):
    start_time = time.time()  
    
    # Cross-validation and get scores (negative MSE is converted to positive RMSE)
    neg_mse_scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
    rmse_scores = np.sqrt(-neg_mse_scores)
    
    # Calculate the average R2 score across all folds
    r2_scores = cross_val_score(model, X, y, cv=cv, scoring='r2', n_jobs=-1)
    
    # Calculate the average MAE score across all folds
    neg_mae_scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
    mae_scores = -neg_mae_scores  # Convert negative MAE to positive

    train_time = time.time() - start_time  

    return {
        'Model': model_name,
        'Mean RMSE': rmse_scores.mean(),
        'Std RMSE': rmse_scores.std(),
        'Mean R2': r2_scores.mean(),
        'Mean MAE': mae_scores.mean(),
        'Std MAE': mae_scores.std(),
        'Training Time (s)': train_time
    }

# Initialize an empty list to store the evaluation results of each model for cross-validation
cv_results = []

# Cross-validation setup (KFold with 5 splits)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate each model using cross-validation for both datasets
for model_name, model in models:
    # Evaluate on the dataset without the outlier
    result_cv = train_and_evaluate_model_cv(model, X, y, kfold, model_name + ' (No Outlier)')
    cv_results.append(result_cv)
    
    # Evaluate on the dataset with the outlier
    result_cv_out = train_and_evaluate_model_cv(model, X_out, y_out, kfold, model_name + ' (With Outlier)')
    cv_results.append(result_cv_out)

# Convert results to a DataFrame
cv_results_df = pd.DataFrame(cv_results)



In [ ]:

#models with fine-tuned parameters
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=450, max_depth=4, learning_rate=0.1, random_state=42)
randomf_model = RandomForestRegressor(n_estimators=500, max_depth=6, random_state=42)
grad_model = GradientBoostingRegressor(n_estimators=450, max_depth=4, learning_rate=0.1, random_state=42)

# Models list
models = [
    ('XGBoost', xgb_model),
    ('Random Forest', randomf_model),
    ('Gradient Boosting', grad_model)
]

In [ ]:
#printing out the result that we got for XGBOOST, RANDOM FOREST AND GRADIENT BOOSTING when we try 
#train_test_split method

print(results_dfi)

In [ ]:
#printing out the result that we got for XGBOOST, RANDOM FOREST AND GRADIENT BOOSTING when we try 
#cross_validation method


print(cv_results_df)

In [ ]:
X_removed_feature = dataset[['AgentCode_encoded', 'ProductCode_encoded', 'ChargeableWeight', 'Pieces', 'FlownMonth_encoded', 'Agent_Total_Weight', 'Product_Avg_Price_per_kg', 'CargoType_encoded']]

X_out_removed_feature = datasetout[['AgentCode_encoded', 'ProductCode_encoded', 'ChargeableWeight', 'Pieces', 'FlownMonth_encoded', 'Agent_Total_Weight', 'Product_Avg_Price_per_kg', 'CargoType_encoded']]


In [ ]:
X_train_rem, X_test_rem, y_train_rem, y_test_rem = train_test_split(X_removed_feature, y, test_size=0.27, 
                                                random_state=42)
X_train_out_rem, X_test_out_rem, y_train_out_rem, y_test_out_rem = train_test_split(X_out_removed_feature, y_out, test_size=0.27, 
                                                random_state=42)

In [ ]:

# Function to train and evaluate a given model and return performance metrics without having to redo it for each model 
def train_and_evaluate_model(model, X_train_rem, y_train_rem, X_test_rem, y_test_rem, model_name):
    # Start measuring training time
    start_time = time.time()  
    
    # Train the provided model on training data
    model.fit(X_train, y_train)  
    
    # Calculate the total time taken to train the model
    train_time = time.time() - start_time  
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)  

    # Calculate evaluation metrics: 
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Return the results as a dictionary for easy appending to the summary list
    return {
        'Model': model_name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'Training Time (s)': train_time
    }

# Initialize an empty list to store the evaluation results of each model
results = []

# Example models to evaluate
models = [
    ('XGBoost', xgb.XGBRegressor(n_estimators=450, max_depth=4, learning_rate=0.1, random_state=42)),
    ('Random Forest', RandomForestRegressor(n_estimators=500, max_depth=6, random_state=42)),
    ('Gradient Boosting', GradientBoostingRegressor(n_estimators=450, max_depth=4, learning_rate=0.1, random_state=42))
]

# Evaluate each model on both datasets
for model_name, model in models:
    # Evaluate on the dataset without the outlier
    result = train_and_evaluate_model(model, X_train_rem, y_train_rem, X_test_rem, y_test_rem, model_name + ' (No Outlier)')
    results.append(result)
    
    # Evaluate on the dataset with the outlier
    result_out = train_and_evaluate_model(model, X_train_out, y_train_out, X_test_out, y_test_out, model_name + ' (With Outlier)')
    results.append(result_out)

# Convert results to a DataFrame for easier visualization
results_df_rem = pd.DataFrame(results)


In [ ]:
print(results_df_rem)

In [ ]:
def train_and_evaluate_model_cv(model, X, y, cv, model_name):
    start_time = time.time()  
    
    # Cross-validation and get scores (negative MSE is converted to positive RMSE)
    neg_mse_scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
    rmse_scores = np.sqrt(-neg_mse_scores)
    
    # Calculate the average R2 score across all folds
    r2_scores = cross_val_score(model, X, y, cv=cv, scoring='r2', n_jobs=-1)
    
    # Calculate the average MAE score across all folds
    neg_mae_scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
    mae_scores = -neg_mae_scores  # Convert negative MAE to positive

    train_time = time.time() - start_time  

    return {
        'Model': model_name,
        'Mean RMSE': rmse_scores.mean(),
        'Std RMSE': rmse_scores.std(),
        'Mean R2': r2_scores.mean(),
        'Mean MAE': mae_scores.mean(),
        'Std MAE': mae_scores.std(),
        'Training Time (s)': train_time
    }

# Initialize an empty list to store the evaluation results of each model for cross-validation
cv_results = []

# Cross-validation setup (KFold with 5 splits)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate each model using cross-validation for both datasets
for model_name, model in models:
    # Evaluate on the dataset without the outlier
    result_cv = train_and_evaluate_model_cv(model, X, y, kfold, model_name + ' (No Outlier)')
    cv_results.append(result_cv)
    
    # Evaluate on the dataset with the outlier
    result_cv_out = train_and_evaluate_model_cv(model, X_out_removed_feature, y_out, kfold, model_name + ' (With Outlier)')
    cv_results.append(result_cv_out)

# Convert results to a DataFrame
cv_results_df_rem = pd.DataFrame(cv_results)


In [50]:
print(cv_results_df_rem)

                              Model  Mean RMSE   Std RMSE   Mean R2  Mean MAE  \
0              XGBoost (No Outlier)  10.503596   1.440348  0.952765  2.039115   
1            XGBoost (With Outlier)  39.002811  37.008528  0.669246  4.906595   
2        Random Forest (No Outlier)  14.191151   3.980004  0.911826  3.360044   
3      Random Forest (With Outlier)  51.030833  32.904589  0.345211  7.618718   
4    Gradient Boosting (No Outlier)  10.261367   1.393032  0.955997  1.882719   
5  Gradient Boosting (With Outlier)  55.452343  36.640594 -0.016429  4.555057   

    Std MAE  Training Time (s)  
0  0.206048           0.553916  
1  1.343565           0.516893  
2  0.548016          10.161563  
3  0.979070           8.888461  
4  0.183258           8.994852  
5  1.205982           8.023459  


# EXPLANATION OF THE RESULTS CAN BE FOUND IN THE GIT IN A FILE UNDER THE NAME result_wiremind1.pdf

In [ ]:
# This function tests the performance of the regression model by selecting a random row 
# from the dataset. It extracts the features and the true target value for that row, 
# makes a prediction using the trained model, and calculates the absolute error 
# between the predicted and true values. This helps us understand how well our model is 
# predicting individual instances from the dataset.


In [49]:

# Combine X and y to make it easier to select a random row
dataset = X.copy()
dataset['target'] = y

# Function to select a random row and test the model for regression
def test_random_row_regression(model, X, y):
    # Select a random index from the dataset
    random_index = np.random.randint(0, len(X))
    
    # Extract the random row and its true target value
    random_row = X.iloc[random_index:random_index+1]
    true_value = y.iloc[random_index]
    
    # Make a prediction using the model
    predicted_value = model.predict(random_row)[0]
    
    # Calculate the absolute error (or any other metric you'd like)
    absolute_error = abs(predicted_value - true_value)
    
    print(f"Random Index: {random_index}")
    print(f"True Value: {true_value}")
    print(f"Predicted Value: {predicted_value}")
    print(f"Absolute Error: {absolute_error:.2f}")

# Example usage
test_random_row_regression(model, X, y)


Random Index: 2048
True Value: 60.0
Predicted Value: 60.512152231548015
Absolute Error: 0.51
